In [1]:
from census import Census
from us import states
import pandas as pd
from arcgis import GIS
import arcpy

In [2]:
sections = sdf = pd.DataFrame.spatial.from_featureclass(r"C:\Users\wkjenkins\gis\titlevi\20210119\arcpro_210218\p20\final.gdb\StopSpacing")
sections

,OBJECTID_1,Seq,SignID,StopID,StopAbbr,StopName,OnSt,AtSt,StopPos,PrefTrans,...,GPS_Lon,GPS_Lat,Dist,Field24,OBJECTID,Route,Route_Name,Service_type,RouteAbbre,SHAPE
0,1,1,246,14818,MCTRLP,MALLINCKRODT CENTER,MALLINCKRODT CENTER TRANSIT CENTER,None,MB,YES,...,-90.308863,38.646948,539.646516,None,31,3001,1 Gold,Local,3001,"{""x"": -90.30886299999997, ""y"": 38.646948000000..."
1,2,2,246,15018,FOHOEB,FORSYTH @ WRIGHTON WAY EB,FORSYTH BLVD,WRIGHTON WAY,MB,NO,...,-90.303218,38.646148,214.504462,None,31,3001,1 Gold,Local,3001,"{""x"": -90.30321799999996, ""y"": 38.646148000000..."
2,3,3,246,2732,SKFONB,SKINKER @ FORSYTH NB,N SKINKER BLVD,FORSYTH AVE,FS,NO,...,-90.301028,38.646613,454.976008,None,31,3001,1 Gold,Local,3001,"{""x"": -90.30102799999997, ""y"": 38.646613000000..."
3,4,4,246,14484,SKMINB,SKINKER @ PERSHING NB,N SKINKER BLVD,PERSHING AVE,FS,YES,...,-90.300504,38.650178,115.122044,None,31,3001,1 Gold,Local,3001,"{""x"": -90.30050399999999, ""y"": 38.650178000000..."
4,5,5,246,2638,WASKEB,WATERMAN @ SKINKER EB,WATERMAN BLVD,N SKINKER BLVD,FS,NO,...,-90.300153,38.650840,276.972889,None,31,3001,1 Gold,Local,3001,"{""x"": -90.30015299999997, ""y"": 38.650840000000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7261,7262,15,246,16318,CSFAEB,CURTIS STEINBERG @ FALLING SPRINGS EB,CURTIS STEINBERG DR,FALLING SPRINGS RD,FS,NO,...,-90.169422,38.585717,972.483017,None,20,4522,Sauget Industrial Parkway,Commuter,4522,"{""x"": -90.16942199999994, ""y"": 38.585717000000..."
7262,7263,16,246,16319,SACSNB,SAUGET INDUSTRIAL PKWY @ CURTIS STEINBERG NB,SAUGET INDUSTRIAL PKWY,CURTIS STEINBERG DR,FS,NO,...,-90.159947,38.581610,200.866294,None,20,4522,Sauget Industrial Parkway,Commuter,4522,"{""x"": -90.15994699999999, ""y"": 38.581610000000..."
7263,7264,17,246,16320,SASANB,SAUGET INDUSTRIAL PKWY @ SAUGET INDUSTRIAL PKW...,SAUGET INDUSTRIAL PKWY,SAUGET INDUSTRIAL PKWY,NS,NO,...,-90.158083,38.582902,208.059836,None,20,4522,Sauget Industrial Parkway,Commuter,4522,"{""x"": -90.15808299999998, ""y"": 38.582902000000..."
7264,7265,18,246,16321,SA16EB,1640 SAUGET INDUSTRIAL PKWY EB,SAUGET INDUSTRIAL PKWY,1640 SAUGET INDUSTRIAL PKWY,MB,NO,...,-90.156640,38.582127,746.271827,None,20,4522,Sauget Industrial Parkway,Commuter,4522,"{""x"": -90.15663999999998, ""y"": 38.582127000000..."


In [75]:
Length_table = sections[['RouteAbbr', 'Passing', 'Length', 'NewDir']].groupby(['RouteAbbr', 'Passing'], as_index=False).agg({'Length': 'sum', 'NewDir': 'count'})
Length_table

,RouteAbbr,Passing,Length,NewDir
0,3001,0,3.129564,9
1,3001,1,5.155673,38
2,3002,0,11.163049,31
3,3002,1,9.985551,72
4,3004,0,4.332480,13
...,...,...,...,...
136,4502,2,57.312314,165
137,4512,2,15.995365,13
138,4517,2,35.656138,37
139,4521,2,9.270968,39


In [76]:
Length_table.rename(columns = {'NewDir' : 'Count'}, inplace = True)
Length_table

,RouteAbbr,Passing,Length,Count
0,3001,0,3.129564,9
1,3001,1,5.155673,38
2,3002,0,11.163049,31
3,3002,1,9.985551,72
4,3004,0,4.332480,13
...,...,...,...,...
136,4502,2,57.312314,165
137,4512,2,15.995365,13
138,4517,2,35.656138,37
139,4521,2,9.270968,39


In [77]:
pass_df = Length_table.loc[Length_table['Passing'] == 1]
fail_df = Length_table.loc[Length_table['Passing'] == 0]
fail_df

,RouteAbbr,Passing,Length,Count
0,3001,0,3.129564,9
2,3002,0,11.163049,31
4,3004,0,4.332480,13
6,3005,0,1.047415,3
8,3008,0,8.017312,27
...,...,...,...,...
120,3514,0,3.729233,10
122,3515,0,5.597535,14
124,3516,0,13.542109,36
126,3517,0,4.196051,10


In [78]:
merge_df = pass_df.merge(fail_df, how = 'inner', left_on='RouteAbbr', right_on='RouteAbbr', suffixes=('_pass', '_fail'))

In [79]:
merge_df

,RouteAbbr,Passing_pass,Length_pass,Count_pass,Passing_fail,Length_fail,Count_fail
0,3001,1,5.155673,38,0,3.129564,9
1,3002,1,9.985551,72,0,11.163049,31
2,3004,1,18.435464,124,0,4.332480,13
3,3005,1,3.223800,27,0,1.047415,3
4,3008,1,10.765273,66,0,8.017312,27
...,...,...,...,...,...,...,...
59,3514,1,11.063573,95,0,3.729233,10
60,3515,1,12.897514,106,0,5.597535,14
61,3516,1,27.431080,242,0,13.542109,36
62,3517,1,22.796604,277,0,4.196051,10


In [80]:
merge_df['p_length_passing'] = merge_df['Length_pass'] / (merge_df['Length_fail'] + merge_df['Length_pass'])
merge_df['p_length_failing'] = 1- merge_df['p_length_passing'] 
merge_df['p_section_passing'] = merge_df['Count_pass'] / (merge_df['Count_fail'] + merge_df['Count_pass'])
merge_df['p_section_failing'] = 1- merge_df['p_section_passing'] 


In [83]:
merge_df.to_csv(os.path.join(os.getcwd(), 'StopSpacing_agg.csv'))